# Task 1 f, Calculating reserve factors against column buckling using Euler-Johnson

## We need from the data file:
- volume of the element
- Iyy calculated with `height_str, width_str, thickness_web, thickness_flange, thickness_skin, stringer_pitch`
- dimensions
- `EulerJohnson(EModulus, I_y, area, length, height_str, thickness_flange, thickness_web, radius, sigma_yield, sigma_applied, c=1)`

## Please enter your name (daniel, fabian, felix, yannis)

In [1]:
name = 'yannis'

## Imports

In [2]:
import pandas as pd
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *
from formulas.helpers import *

## Constants

In [ ]:
if name == 'yannis':
    sigma_yield = 490
    EModulus = 65241.07 
elif name == 'fabian':
    sigma_yield = 490
    EModulus = 65420.46
elif name == 'daniel':
    sigma_yield = 490
    EModulus = 65669.47
elif name == 'felix':
    sigma_yield = 490
    EModulus = 65143.57

stringer_pitch = 200
effective_width = stringer_pitch/2

height_str = 40
width_str = 45
thickness_web = 10
thickness_flange = 10
thickness_skin = 10

## File selection

In [4]:
#Import the stress of the panels  
paneldf = pd.read_excel(f'../data/{name}/panel_properties.xlsx', index_col=0)
print(paneldf.head(10))

          shortestside  thickness  volume  longestside
elements                                              
1                  200          4  200000          250
2                  200          4  200000          250
3                  200          4  200000          250
4                  200          4  200000          250
5                  200          4  200000          250
6                  200          4  200000          250
7                  200          4  200000          250
8                  200          4  200000          250
9                  200          4  200000          250
10                 200          4  200000          250


In [5]:
paneldf['Iyy'] = paneldf.apply(crosssectional_properties_tee_skin_row, axis=1)

KeyError: 'height_str'

## Calculate Iyy

In [ ]:
res = crosssectional_properties_tee_skin(height_str, width_str, thickness_web, thickness_flange, thickness_skin, effective_width)
res

(1750.0, 245440.48)